In [1]:
import findspark
findspark.init()

In [2]:
import yaml
import os

import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
from pyspark.sql.types import StructType
from pyspark.sql.functions import col, to_date, to_timestamp, explode
from pyspark.sql.utils import AnalysisException
from pyspark.errors import PySparkException
from pymongo import MongoClient
from pymongo import errors

In [3]:
def gen_mongo_uri(mongo_conf) -> str | None:
    """
    generate mongo connection uri based on input
    :return:
    """
    if not mongo_conf:
        return None
    return (f"mongodb+srv://{mongo_conf['user']}:"
            f"{mongo_conf['token']}@"
            f"{mongo_conf['host']}"
            f"/?retryWrites=true&w=majority")

def get_property(conf) -> dict:
    """
    get property for database
    """
    return {key: conf[key] for key in conf.keys()
                         & {'user', 'password', 'driver'}}

def gen_maria_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None

    db = conf['database'] if not db else db
    return (f"jdbc:mysql://{conf['host']}:"
            f"{conf['port']}/"
            f"{db}?permitMysqlScheme")

def gen_postgres_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:postgresql://{conf['host']}:"
            f"{conf['port']}/{db}")

def gen_mssql_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for maria database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;trustServerCertificate=true;")

def gen_azure_jdbc(conf, db: str = '') -> str | None:
    """
    get connection jdbc string for azure sql database
    """
    if not conf:
        return None
    db = conf['database'] if not db else db
    return (f"jdbc:sqlserver://{conf['host']}:{conf['port']};"
            f"databaseName={db};encrypt=true;trustServerCertificate=true;")


def init_mongodb_client(uri: str) -> MongoClient | None:
    """
    initialize the mongo client
    """
    try:
        # Initialize new MongoDB client
        client = MongoClient(uri)
    except errors.ConnectionFailure as e:
        # Handle connection failure gracefully
        print(f"Failed to connect to MongoDB: {e}")

        return None
    else:
        return client

def prepare_dataframe(spark, sc, data) -> DataFrame:
    """
    prepare dataframe when the data might be in different data types
    :param spark: spark session
    :param sc: spark context
    :param data: data to be processed
    :return: spark sql dataframe
    """
    if isinstance(data, pd.DataFrame) and not data.empty:
        df = spark.createDataFrame(data)
    elif isinstance(data, DataFrame) and not data.isEmpty():
        df = data
    elif isinstance(data, list):
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    elif isinstance(data, dict):
        data = [data]
        df = spark.read.json(sc.parallelize([json.dumps(record) for record in data]))
    else:
        # initialize empty dataframe
        schema = StructType([])
        df = spark.createDataFrame([], schema)

    return df
    
def write_to_mongo(spark, data, 
                   uri: str, db: str, col: str) -> None:
    """
    write data into mongo db
    :param spark: sparkSession
    :param data: 
    :param uri: str, 
    :param db: str, 
    :param col: str, 
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    config = {
        'uri': uri,
        'database': db,
        'collection': col
    }
    if not df.isEmpty():
        df.write.format("mongo") \
            .options(**config) \
            .mode("append") \
            .save()

def write_to_database(spark, data, conf,
                   db: str, write_table: str, type: str = 'mariadb') -> None:
    """
    write data into database table
    :param data:
    :param uri:
    :param db:
    :param table:
    :param type:
    :return:
    """
    sc = spark.sparkContext
    df = prepare_dataframe(spark, sc, data)
    properties = get_property(conf)

    jdbc = ''
    if type == 'mariadb':
        jdbc = gen_maria_jdbc(conf, db)
    elif type == 'postgres':
        jdbc = gen_postgres_jdbc(conf, db)
    elif type == 'mssql':
        jdbc = gen_mssql_jdbc(conf, db)
    elif type == 'azure':
        jdbc = gen_azure_jdbc(conf, db)
    else:
        raise ValueError('error when generating jdbc')

    if not df.isEmpty():
        df.write.jdbc(
            url=jdbc,
            table=write_table,
            mode="append",
            properties=properties
        )
    else:
        print('empty dataset')

def sentiment_analysis(data, tokenizer, model) -> pd.DataFrame:
    """
    sentiment analysis and retrun dataframe with score
    """
    data['sentiment'] = None
    for idx, text in enumerate(data['text']):
        tokenized_news = tokenizer(text, truncation=True, return_tensors="tf")
        logits = model.predict(tokenized_news).logits
        probabilities = tf.nn.softmax(logits)
        data.at[idx, 'sentiment'] = probabilities.numpy()
    return data

In [4]:
with open('conf.yaml', 'r') as file:
    config = yaml.safe_load(file)

mongo_uri = gen_mongo_uri(config['mongodb'])

In [5]:
try:
    # getting the spark instance
    spark = SparkSession.builder \
        .appName('Big Data Project ETL') \
        .config("spark.mongodb.input.uri", mongo_uri) \
        .config("spark.mongodb.output.uri", mongo_uri) \
        .config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector_2.12:1.2.0,"
                                       "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .master("local[*]") \
        .getOrCreate()
except PySparkException as e:
    print(f"Failed to get or create Spark: {e}")
except Exception as e:
    print(f"Exception Caught: {e}")

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sutring/.ivy2/cache
The jars for the packages stored in: /home/sutring/.ivy2/jars
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ce3c68d6-5d57-462a-b0ac-188c1e0c1e53;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 271ms :: artifacts dl 15ms
	:: modules in use:
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.mongodb#bson;4.0.5 from central

In [6]:
# read historical data with sentiment from mariadb
historical_stock_df = spark.read.jdbc(
    url=gen_maria_jdbc(config['mariadb'], 'finance_out'),
    table='historical_with_sentiment',
    properties=get_property(config['mariadb'])
)
# read commitment trader report data from mssql
cot_report_df = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'finance_api'),
    table='fmp_cmtmt_trader_report',
    properties=get_property(config['postgres'])
)
# read income statement data from postgresql
income_stmt_df = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'finance_api'),
    table='fmp_income_stmt',
    properties=get_property(config['postgres'])
)

In [7]:
historical_stock_df.show(5), historical_stock_df.count()

+------+----------+-------------------+----------+---------+
|symbol|      date|              Close|    Volume|sentiment|
+------+----------+-------------------+----------+---------+
|  AAPL|2004-03-03|0.42714300751686096| 225131200|     NULL|
|  AAPL|2004-03-04|0.44928601384162903| 660223200|     NULL|
|  AAPL|2004-03-05|0.47749999165534973|1540599200|     NULL|
|  AAPL|2004-03-08| 0.4642859995365143| 522872000|     NULL|
|  AAPL|2004-03-09|0.48392900824546814| 618363200|     NULL|
+------+----------+-------------------+----------+---------+
only showing top 5 rows



(None, 116277)

In [8]:
cot_report_df.show(5), cot_report_df.count()

+-------------------+------+--------------------------+---------------------------+-----------------------+------------------------+-----------+---------------+
|               date|symbol|noncomm_positions_long_all|noncomm_positions_short_all|comm_positions_long_all|comm_positions_short_all|comm_spread|non_comm_spread|
+-------------------+------+--------------------------+---------------------------+-----------------------+------------------------+-----------+---------------+
|2024-02-27 00:00:00|    BT|                     20034|                      22001|                   1759|                     799|        960|          -1967|
|2024-02-20 00:00:00|    BT|                     19624|                      21722|                   1862|                     660|       1202|          -2098|
|2024-02-13 00:00:00|    BT|                     19147|                      21068|                   1514|                     362|       1152|          -1921|
|2024-02-06 00:00:00|    BT|      

(None, 9775)

In [9]:
income_stmt_df.show(5), income_stmt_df.count()

24/03/10 18:00:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+------------+----------+---------------+-------------+----------+---------------------------+---------+------------+------+----------+-----------+--------------------+--------------------------------+-----------+----------------+---------------+--------------------+----------------+---------------+--------------+--------------------+---------+--------------+-----------------+---------------+--------------------+-------------+------+----------------+------------------------------+----------+---------------------------+---------------------------------------+------+---------------------------+---------------------+------------------------+
|       acceptedDate|calendarYear|       cik|costAndExpenses|costOfRevenue|      date|depreciationAndAmortization|   ebitda| ebitdaratio|   eps|epsdiluted|fillingDate|           finalLink|generalAndAdministrativeExpenses|grossProfit|grossProfitRatio|incomeBeforeTax|incomeBeforeTaxRatio|incomeTaxExpense|interestExpense|interestIncom

(None, 3026)

In [10]:
# get unique symbols
stock_symbols = historical_stock_df.select("symbol").distinct().rdd.flatMap(lambda x : x).collect()
sec_symbols = cot_report_df.select("symbol").distinct().rdd.flatMap(lambda x : x).collect()

In [11]:
# save historical data into postgresql
for symbol in stock_symbols:
    stock_data = historical_stock_df.filter(historical_stock_df['symbol'] == symbol) \
                                .select('date', 'Close', 'sentiment') \
                                .withColumnRenamed('Close', symbol)
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, stock_data, config['postgres'],
                          db='local_finance_stage', write_table=f'stock_{symbol}', type='postgres')
    except Exception as e:
        print(e)

In [12]:
# save commitment report data into postgresql
for symbol in sec_symbols:
    cot_data = cot_report_df.filter(cot_report_df['symbol'] == symbol) \
                            .withColumn('date', to_date('date')) \
                            .select("date", "symbol", "comm_spread", "non_comm_spread")
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, cot_data, config['postgres'],
                          db='local_finance_stage', write_table=f'cot_report_{symbol}', type='postgres')
    except Exception as e:
        print(e)

In [13]:
# save income statement data into postgresql
for symbol in stock_symbols:
    income_stmt_data = income_stmt_df.filter(income_stmt_df['symbol'] == "AAPL") \
                                        .drop('date') \
                                        .withColumn('acceptedDate', 
                                                    income_stmt_df.acceptedDate.cast("date").alias('date')) \
                                        .withColumnRenamed('acceptedDate', 'date')
    try:
        # save with 'stock_' as prefix
        write_to_database(spark, income_stmt_data, config['postgres'],
                          db='local_finance_stage', write_table=f'income_stmt_{symbol}', type='postgres')
    except Exception as e:
        print(e)

In [17]:
company_profile = spark.read.csv("hdfs://localhost:9000/user/input/project/fmp_company_profile", header='true')

In [19]:
# save company profile in postgresql
try:
    write_to_database(spark, company_profile, config['postgres'],
                      db='local_finance_stage', write_table=f'stock_company_profile', type='postgres')
except Exception as e:
    print(e)

In [20]:
# load state crime data from postgresql
# state_crime_rate_df = spark.read.jdbc(
#     url=gen_postgres_jdbc(config['postgres'], 'finance_out'),
#     table='state_crime_rate',
#     properties=get_property(config['postgres'])
# )

In [28]:
# save state crime rate in mssql or azure sql
# try:
#     # save with 'stock_' as prefix
#     write_to_database(spark, state_crime_rate_df, config['postgres'],
#                       db='local_finance_stage', write_table='state_crime_rate', type='postgres')
# except Exception as e:
#     print(e)

In [21]:
# load inside transactions from mongodb
inside_trading_df = spark.read.format('mongo') \
    .option("database", 'finance_api') \
    .option("collection", 'finn_insider_transactions') \
    .load()

inside_trading_df.show(5)

+--------------------+--------------------+------+
|                 _id|                data|symbol|
+--------------------+--------------------+------+
|{65e550a0b142680e...|[{-100000, , 2024...|  AAPL|
|{65e550bab142680e...|[{-300, , 2024-02...|  MSFT|
|{65e550d4b142680e...|[{-5000, , 2024-0...|  NVDA|
|{65e550ecb142680e...|[{-31493, , 2024-...|  META|
|{65e55104b142680e...|[{-500, , 2024-02...|  AMZN|
+--------------------+--------------------+------+
only showing top 5 rows



In [22]:
inside_trading_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- change: long (nullable = true)
 |    |    |-- currency: string (nullable = true)
 |    |    |-- filingDate: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- isDerivative: boolean (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- share: long (nullable = true)
 |    |    |-- source: string (nullable = true)
 |    |    |-- symbol: string (nullable = true)
 |    |    |-- transactionCode: string (nullable = true)
 |    |    |-- transactionDate: string (nullable = true)
 |    |    |-- transactionPrice: double (nullable = true)
 |-- symbol: string (nullable = true)



In [23]:
# flattern inside_trading schema
inside_trading_new_df = inside_trading_df.select('symbol', explode('data').alias('trade_info')) \
                                    .select('symbol', 'trade_info.transactionDate', 
                                            'trade_info.share', 'trade_info.name',
                                            'trade_info.transactionPrice', 'trade_info.change')
# df_flat = df_flat.select('name', 'age', 'car_info.car1', 'car_info.model')

inside_trading_new_df.show(15)

+------+---------------+-------+-----------------+----------------+-------+
|symbol|transactionDate|  share|             name|transactionPrice| change|
+------+---------------+-------+-----------------+----------------+-------+
|  AAPL|     2024-02-29|4434576|LEVINSON ARTHUR D|          180.94|-100000|
|  AAPL|     2024-02-28|   1516|     WAGNER SUSAN|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|   SUGAR RONALD D|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|  LOZANO MONICA C|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|LEVINSON ARTHUR D|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|      JUNG ANDREA|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|      Gorsky Alex|             0.0|   1516|
|  AAPL|     2024-02-28|   1516|   Austin Wanda M|             0.0|   1516|
|  AAPL|     2024-02-01|      0|     WAGNER SUSAN|             0.0|  -1852|
|  AAPL|     2024-02-01|  60975|     WAGNER SUSAN|             0.0|   1852|
|  AAPL|    

In [24]:
inside_trading_new_df.count()

41787

In [25]:
# save inside trading in postgresql
try:
    write_to_database(spark, inside_trading_new_df, config['postgres'],
                      db='local_finance_stage', write_table=f'stock_inside_trading', type='postgres')
except Exception as e:
    print(e)

In [ ]:
#save housing data into postgresql


In [6]:
# filling null values in back fill way
housing_ps = pd.read_csv('data/House_Price_month.csv')
housing_ps.drop(columns=['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName'], inplace=True)
housing_ps = housing_ps.T.bfill().T

In [7]:
housing_df = spark.createDataFrame(housing_ps)

In [8]:
# save stage housing price in postgresql
try:
    write_to_database(spark, housing_df, config['postgres'],
                      db='local_finance_stage', write_table=f'state_housing_price', type='postgres')
except Exception as e:
    print(e)

In [11]:
print(stock_symbols)

['AAPL', 'LOGI', 'XOM', 'BMY', 'META', 'LNG', 'TSLA', 'EME', 'SWN', 'AXTA', 'CL', 'SBLK', 'CVX', 'TNDM', 'GTLB', 'GOOGL', 'APA', 'ON', 'MOS', 'DBD', 'DSGX', 'KOP', 'BTU', 'THC', 'AMZN', 'MSFT', 'CRSR', 'DNOW', 'NVDA']


In [12]:
# transfer tables from local respository to azure sql
# transfer historical data and income statement
for symbol in stock_symbols:
    print(f'{"-" * 20} processing {symbol} {"-" * 20}')
    income_stmt = spark.read.jdbc(
        url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
        table=f'income_stmt_{symbol}',
        properties=get_property(config['postgres'])
    )
    try:
        write_to_database(spark, income_stmt, config['azuresql'],
                          db='final_finance_stage', write_table=f'income_stmt_{symbol}', type='azure')
    except Exception as e:
        print(e)
        
    stock_data = spark.read.jdbc(
        url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
        table=f'stock_{symbol}',
        properties=get_property(config['postgres'])
    )
    try:
        write_to_database(spark, stock_data, config['azuresql'],
                          db='final_finance_stage', write_table=f'stock_{symbol}', type='azure')
    except Exception as e:
        print(e)
    print(f'{"-" * 20} {symbol} complete {"-" * 20}')
    

-------------------- processing AAPL --------------------
-------------------- AAPL complete --------------------
-------------------- processing LOGI --------------------
-------------------- LOGI complete --------------------
-------------------- processing XOM --------------------
-------------------- XOM complete --------------------
-------------------- processing BMY --------------------
-------------------- BMY complete --------------------
-------------------- processing META --------------------
-------------------- META complete --------------------
-------------------- processing LNG --------------------


-------------------- LNG complete --------------------
-------------------- processing TSLA --------------------
-------------------- TSLA complete --------------------
-------------------- processing EME --------------------
-------------------- EME complete --------------------
-------------------- processing SWN --------------------
-------------------- SWN complete --------------------
-------------------- processing AXTA --------------------
-------------------- AXTA complete --------------------
-------------------- processing CL --------------------
-------------------- CL complete --------------------
-------------------- processing SBLK --------------------
-------------------- SBLK complete --------------------
-------------------- processing CVX --------------------
-------------------- CVX complete --------------------
-------------------- processing TNDM --------------------
-------------------- TNDM complete --------------------
-------------------- processing GTLB ------

In [13]:
# transfer commitment trader report
for symbol in sec_symbols:
    print(f'{"-" * 20} processing {symbol} {"-" * 20}')
    cot_data = spark.read.jdbc(
        url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
        table=f'cot_report_{symbol}',
        properties=get_property(config['postgres'])
    )
    try:
        write_to_database(spark, cot_data, config['azuresql'],
                          db='final_finance_stage', write_table=f'cot_report_{symbol}', type='azure')
    except Exception as e:
        print(e)
    print(f'{"-" * 20} {symbol} complete {"-" * 20}')

-------------------- processing ZN --------------------
-------------------- ZN complete --------------------
-------------------- processing DX --------------------
-------------------- DX complete --------------------
-------------------- processing CL --------------------
-------------------- CL complete --------------------
-------------------- processing ES --------------------
-------------------- ES complete --------------------
-------------------- processing GC --------------------
-------------------- GC complete --------------------
-------------------- processing LS --------------------
-------------------- LS complete --------------------
-------------------- processing BT --------------------
-------------------- BT complete --------------------
-------------------- processing SI --------------------
-------------------- SI complete --------------------
-------------------- processing NG --------------------
-------------------- NG complete --------------------


In [14]:
# transfer company profile data
company_profile = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
    table=f'stock_company_profile',
    properties=get_property(config['postgres'])
)
try:
    write_to_database(spark, stock_data, config['azuresql'],
                      db='final_finance_stage', write_table=f'stock_company_profile', type='azure')
except Exception as e:
    print(e)

In [15]:
# transfer state crime data
state_crime = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
    table=f'state_crime_rate',
    properties=get_property(config['postgres'])
)
try:
    write_to_database(spark, state_crime, config['azuresql'],
                      db='final_finance_stage', write_table=f'state_crime_rate', type='azure')
except Exception as e:
    print(e)

In [17]:
# transfer statehousing data
state_housing = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
    table=f'state_housing_price',
    properties=get_property(config['postgres'])
)
try:
    write_to_database(spark, state_housing, config['azuresql'],
                      db='final_finance_stage', write_table=f'state_housing_price', type='azure')
except Exception as e:
    print(e)

In [18]:
# transfer stock inside trading
inside_trading = spark.read.jdbc(
    url=gen_postgres_jdbc(config['postgres'], 'local_finance_stage'),
    table=f'stock_inside_trading',
    properties=get_property(config['postgres'])
)
try:
    write_to_database(spark, stock_data, config['azuresql'],
                      db='final_finance_stage', write_table=f'stock_inside_trading', type='azure')
except Exception as e:
    print(e)

In [20]:
spark.stop()